In [1]:
import numpy as np
import scipy.sparse as sp
import torch

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

In [3]:
def feature_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [4]:
def adj_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1)) # Sum each row
    r_inv = np.power(rowsum, -1/2).flatten() # Negative square root
#     r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv) # Create diagonal matrix
    
    # D^(-1/2).A.D^(-1/2)
    mx = r_mat_inv.dot(mx)
    mx = mx.dot(r_mat_inv)
    return mx

In [5]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [6]:
def load_data(path="../data/cora/", dataset="cora"):
    
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}/train_data.txt".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 0:3], dtype=np.float32) # Processing features into a sparse matrix
    labels = encode_onehot(idx_features_labels[:, -2]) # one-hot encoding the labels
    

    # build graph
    idx = np.array(idx_features_labels[:, -1], dtype=np.int32) # Reading node-ids
    
    # Creating node ids to eliminate discrepencies in node ids in the data
    idx_map = {j: i for i, j in enumerate(idx)} # Creating index for nodes to map it in adjacency matrix
    
    edges_unordered = np.genfromtxt("{}{}/train_edges.txt".format(path, dataset),
                                    dtype=np.int32) # Reading edges
    

    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape) # Mapping node-ids in the edge list to the index
    
    # Build adjacency matrix
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)
    
    # CHECK OUT THE DIFFERENCES BETWEEN csr_matrix (features) and coo_matrix (adj)
    
    # Normalizing features
    features = feature_normalize(features)
    
#     # build symmetric adjacency matrix
#     adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

#     Normalizing the adjacency matrix after adding self loops
    adj = adj_normalize(adj + sp.eye(adj.shape[0]))

    # Setting training, validation, and test range
    idx_train = range(430000)
    idx_val = range(450000, 600000)
    idx_test = range(700000, 900000)

    # Converting all matrices into pytorch tensors
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [7]:
# Function to find accuracy from two tensors
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels) # Get the index of maximum value of 1 dimension and typecast to labels datatype
    correct = preds.eq(labels).double() # Convert into double
    correct = correct.sum() # Sum correct predictions
    return correct / len(labels)

In [8]:
import math

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

# Class to define a neural network layer that inherits PyTorch Module
# Check out documentaion of the base class 'Module' at:
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html
class GraphConvolution(Module):
    # Each layer requires no. of input features, no. of output features, and optional bias
    def __init__(self, in_feat, out_feat, bias=True):
        super(GraphConvolution, self).__init__()
        
        self.in_features = in_feat
        self.out_features = out_feat
        
        # Using Parameter to automatically add weights and bias to learnable parameters
        #THIS WILL BE USEFUL ONLY WHEN WE USE Module in the model
        self.weight = Parameter(torch.FloatTensor(in_feat, out_feat))
        
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_feat))
        else:
            self.register_parameter('bias', None)
            
        self.reset_parameters()
        
    # Function to get uniform distribution of weights and bias values
    # Can be removed if necessary
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
            
    # Forward function where it actually requires the input data and operations
    def forward(self, inp, adj):
        # Basically we multiply A.H,W
        support = torch.mm(inp, self.weight)
        output = torch.spmm(adj, support)
        
        # Adding bias if true
        if self.bias is not None:
            return output + self.bias
        else:
            return output

In [9]:
import torch.nn as nn
import torch.nn.functional as F

# Class to define the model architecture
class GCN(nn.Module):
    # The model needs no. of input features, no. of hidden units,
    # no. of classes, and optional dropout
    
    # NOTE: We use a simply model with one hidden layer
        # Architecture will change for deep models
        # Ideally, we keep only a few layers in most GNNs
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        
        # Defining one hidden layer and one output layer
        self.gcn1 = GraphConvolution(nfeat, nhid)
        self.gcn2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout
        
    # Similar to GraphConvolution, we give required input data to the forward function
    # And specify operations - here it is activation and dropout
    def forward(self, x, adj):
        x = F.relu(self.gcn1(x, adj)) # Applying non-linearity on hidden layer 1
        # Checkout difference between nn.Dropout() and F.dropout()
        x = F.dropout(x, self.dropout, training=self.training) 
        x = self.gcn2(x, adj)
        return F.log_softmax(x, dim=1) # Applying lograthmic softmax on output layer

In [10]:
import torch.optim as optim

adj, features, labels, train_ids, val_ids, test_ids = load_data('/home/lvaughan/Pileup/data/','ZZ4nu')
# load_data('/home/abagava/Datasets/Physics/','Primary')

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)

# Using Adam optimizer. Other optimizer can be used too
optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)

Loading ZZ4nu dataset...


/tmp/ipykernel_468257/3650411315.py:8: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:605.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [11]:
# # Code for GPU computing

# # CHANGE THIS CODE TO SUIT YOUR VERSION OF PYTORCH. THE SYNTAX OF THIS COULD VARY SIGNIFICANTLY

# # If cuda is available, movie all data to gpu
# # And preparing for CUDA operations
#if torch.cuda.is_available():
#     model.cuda()
#     features = features.cuda()
#     adj = adj.cuda()
#     labels = labels.cuda()
#     train_ids.cuda()
#     val_ids.cuda()
#     test_ids.cuda()

In [12]:
import time

def train(epoch):
    t = time.time()
    
    # Evaluating the model in training mode
    model.train()
    optimizer.zero_grad() # Reseting gradient at each layer to avoid exploding gradient problem
    output = model(features, adj)
     
        
    # Optimizing with nll_loss. Other losses like cross_entropy can also be used
    loss_train = F.nll_loss(output[train_ids], labels[train_ids])
    acc_train = accuracy(output[train_ids], labels[train_ids])
    # backprop and optimize model parameters
    # Not needed to specify parameters when using Parameter
    loss_train.backward()
    optimizer.step()
    
    # Evaluating validation performance separately
    model.eval()
    output = model(features, adj)
    loss_val = F.nll_loss(output[val_ids], labels[val_ids])
    acc_val = accuracy(output[val_ids], labels[val_ids])


    loss_val = F.nll_loss(output[val_ids], labels[val_ids])
    acc_val = accuracy(output[val_ids], labels[val_ids])

    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [13]:
# Function to test the model
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[test_ids], labels[test_ids])
    acc_test = accuracy(output[test_ids], labels[test_ids])
    
    
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [14]:
t_total = time.time()

for epoch in range(400):
    train(epoch)
    print('Iteraion-' + str(epoch))
    print('Total time elapsed: {:.4f}s'.format(time.time() - t_total))
    print('\n')

print()
print('Optimization finished...')
print('Total time elapsed: {:.4f}s'.format(time.time() - t_total))

test()

Epoch: 0001 loss_train: 0.7601 acc_train: 0.5287 loss_val: 0.7143 acc_val: 0.4305 time: 0.8108s
Iteraion-0
Total time elapsed: 0.8164s


Epoch: 0002 loss_train: 0.7340 acc_train: 0.5414 loss_val: 0.6960 acc_val: 0.4558 time: 0.8054s
Iteraion-1
Total time elapsed: 1.6281s


Epoch: 0003 loss_train: 0.7189 acc_train: 0.5067 loss_val: 0.6865 acc_val: 0.5695 time: 0.7932s
Iteraion-2
Total time elapsed: 2.4277s


Epoch: 0004 loss_train: 0.7108 acc_train: 0.4672 loss_val: 0.6840 acc_val: 0.5695 time: 0.8139s
Iteraion-3
Total time elapsed: 3.2474s


Epoch: 0005 loss_train: 0.7088 acc_train: 0.4520 loss_val: 0.6853 acc_val: 0.5695 time: 0.8066s
Iteraion-4
Total time elapsed: 4.0603s


Epoch: 0006 loss_train: 0.7109 acc_train: 0.4544 loss_val: 0.6876 acc_val: 0.5695 time: 0.7853s
Iteraion-5
Total time elapsed: 4.8520s


Epoch: 0007 loss_train: 0.7128 acc_train: 0.4641 loss_val: 0.6890 acc_val: 0.5695 time: 0.8176s
Iteraion-6
Total time elapsed: 5.6751s


Epoch: 0008 loss_train: 0.7134 acc_train:

Epoch: 0061 loss_train: 0.6799 acc_train: 0.5470 loss_val: 0.6777 acc_val: 0.5695 time: 0.8167s
Iteraion-60
Total time elapsed: 49.7648s


Epoch: 0062 loss_train: 0.6796 acc_train: 0.5479 loss_val: 0.6774 acc_val: 0.5695 time: 0.8320s
Iteraion-61
Total time elapsed: 50.6034s


Epoch: 0063 loss_train: 0.6789 acc_train: 0.5485 loss_val: 0.6771 acc_val: 0.5695 time: 0.8138s
Iteraion-62
Total time elapsed: 51.4248s


Epoch: 0064 loss_train: 0.6791 acc_train: 0.5489 loss_val: 0.6768 acc_val: 0.5695 time: 0.8160s
Iteraion-63
Total time elapsed: 52.2473s


Epoch: 0065 loss_train: 0.6784 acc_train: 0.5498 loss_val: 0.6765 acc_val: 0.5695 time: 0.8233s
Iteraion-64
Total time elapsed: 53.0780s


Epoch: 0066 loss_train: 0.6784 acc_train: 0.5493 loss_val: 0.6761 acc_val: 0.5695 time: 0.8146s
Iteraion-65
Total time elapsed: 53.9002s


Epoch: 0067 loss_train: 0.6780 acc_train: 0.5499 loss_val: 0.6758 acc_val: 0.5695 time: 0.8063s
Iteraion-66
Total time elapsed: 54.7131s


Epoch: 0068 loss_train: 0.6

Epoch: 0120 loss_train: 0.6479 acc_train: 0.5395 loss_val: 0.6412 acc_val: 0.5695 time: 0.7950s
Iteraion-119
Total time elapsed: 98.1064s


Epoch: 0121 loss_train: 0.6467 acc_train: 0.5358 loss_val: 0.6400 acc_val: 0.5695 time: 0.7949s
Iteraion-120
Total time elapsed: 98.9071s


Epoch: 0122 loss_train: 0.6456 acc_train: 0.5342 loss_val: 0.6386 acc_val: 0.5695 time: 0.7999s
Iteraion-121
Total time elapsed: 99.7125s


Epoch: 0123 loss_train: 0.6445 acc_train: 0.5348 loss_val: 0.6375 acc_val: 0.5695 time: 0.8077s
Iteraion-122
Total time elapsed: 100.5278s


Epoch: 0124 loss_train: 0.6435 acc_train: 0.5346 loss_val: 0.6365 acc_val: 0.5695 time: 0.8150s
Iteraion-123
Total time elapsed: 101.3487s


Epoch: 0125 loss_train: 0.6428 acc_train: 0.5316 loss_val: 0.6349 acc_val: 0.5695 time: 0.8062s
Iteraion-124
Total time elapsed: 102.1626s


Epoch: 0126 loss_train: 0.6413 acc_train: 0.5294 loss_val: 0.6338 acc_val: 0.5695 time: 0.8031s
Iteraion-125
Total time elapsed: 102.9718s


Epoch: 0127 loss

Epoch: 0179 loss_train: 0.5764 acc_train: 0.6774 loss_val: 0.5533 acc_val: 0.7370 time: 0.7904s
Iteraion-178
Total time elapsed: 145.9046s


Epoch: 0180 loss_train: 0.5749 acc_train: 0.6799 loss_val: 0.5521 acc_val: 0.7375 time: 0.8034s
Iteraion-179
Total time elapsed: 146.7137s


Epoch: 0181 loss_train: 0.5745 acc_train: 0.6796 loss_val: 0.5511 acc_val: 0.7383 time: 0.7943s
Iteraion-180
Total time elapsed: 147.5138s


Epoch: 0182 loss_train: 0.5731 acc_train: 0.6825 loss_val: 0.5500 acc_val: 0.7383 time: 0.7930s
Iteraion-181
Total time elapsed: 148.3119s


Epoch: 0183 loss_train: 0.5722 acc_train: 0.6823 loss_val: 0.5490 acc_val: 0.7390 time: 0.8075s
Iteraion-182
Total time elapsed: 149.1244s


Epoch: 0184 loss_train: 0.5719 acc_train: 0.6827 loss_val: 0.5480 acc_val: 0.7393 time: 0.8005s
Iteraion-183
Total time elapsed: 149.9326s


Epoch: 0185 loss_train: 0.5714 acc_train: 0.6823 loss_val: 0.5470 acc_val: 0.7400 time: 0.8081s
Iteraion-184
Total time elapsed: 150.7458s


Epoch: 0186 l

In [15]:
# FOLLOWING SET OF CODE IS TO UNDERSTAND THE DATA
# IT IS NOT PART OF GCN

In [16]:
# Sample program to understand the data
# phyData_path = '/home/abagava/Datasets/Physics/Primary/'
# phyData_train_edges = 'train_edges.txt'
# phyData_train_feat = 'train_data.txt'
# phyData_test_edges = 'test_edges.txt'
# phyData_test_feat = 'test_data.txt'

# train_nodes = set()
# test_nodes = set()

# for i,line in enumerate(open(phyData_path + phyData_train_edges, 'r')):
#     edges = line.split()
#     train_nodes.update(edges)
    
# print('Train data contains ' + str(i) + ' edges.')
# print('Validation is 30% ' + str(0.3*i) + ' edges.')

In [17]:
# nodes_at_train = set()

# for i,line in enumerate(open(phyData_path + phyData_test_edges, 'r')):
#     edges = line.split()
    
#     for node in edges:
#         if node in train_nodes:
#             nodes_at_train.add(node)
    
#     test_nodes.update(edges)
    
# print('Test data contains ' + str(i) + ' edges.')

In [18]:
# print('Number of training nodes: ' + str(len(train_nodes)))
# print('Number of test nodes: ' + str(len(test_nodes)))
# print('No. of test nodes present in training: ' + str(len(nodes_at_train)))

In [19]:
# fp = open(phyData_path + phyData_train_feat, 'r')

# feat_nodes = set()

# present = 0
# not_present = 0

# for i, line in enumerate(fp):
#     feat_nodes.add(line.split()[-1])
# fp.close()


# # Adding 0's as features for nodes that lack features

# fp = open(phyData_path + phyData_train_feat, 'a')
# for node in train_nodes:
#     if node not in feat_nodes:
#         fp.write('0.0 0.0 0.0 0 ' + str(node)+' \n')
# #     if node in feat_nodes:
# #         present += 1
# #     else:
# #         not_present += 1
# fp.close()

# print(present)
# print(not_present)

In [20]:
# fp = open(phyData_path + 'train_data-Copy1.txt', 'r')

# for i, line in enumerate(fp):
#     feat_nodes.add(line.split()[-1])
# fp.close()

# for node in train_nodes:
#     if node in feat_nodes:
#         present += 1
#     else:
#         not_present += 1
    
# print(present)
# print(not_present)

### 